# 1. Importación datasets

In [1]:
import pandas as pd

import joblib

In [2]:
df_procesado = pd.read_csv('df_procesado2.csv')

In [3]:
airport_station_mapping = pd.read_csv('airport_station_mapping.csv')

In [4]:
# 13751 in airport_station_mapping.STATION_ID.values

# Estaciones con WBAN diferente
# [99999: 93073, 12846: 12834]

In [ ]:
modelo_visibilidad = joblib.load(r"Base_climaNOAA\modelo_visibilidad.joblib")

In [ ]:
df_99999 = pd.read_excel(r'Base_climaNOAA\Faltantes\ASPEN_PITKIN_CO_SARD_Consolidado.xlsx',
                         sheet_name='Hoja1')

In [ ]:
df_11640 = pd.read_excel(r'Base_climaNOAA\Faltantes\CYRIL E KING AIRPORT_Consolidado.xlsx',
                         sheet_name='Hoja1')

In [ ]:
df_12846 = pd.read_excel(r'Base_climaNOAA\Faltantes\DAYTONA BEACH INTERNATIONAL AIRPORT_Consolidado.xlsx',
                         sheet_name='Hoja1')

In [ ]:
df_41415 = pd.read_excel(r'Base_climaNOAA\Faltantes\GUAM WSMO_Consolidado.xlsx',
                         sheet_name='Hoja1')

In [ ]:
df_11624 = pd.read_excel(r'Base_climaNOAA\Faltantes\HENRY E ROHLSEN AIRPORT_Consolidado.xlsx',
                         sheet_name='Hoja1')

# 2. Funciones

## 2.1 Estandariza columnas

In [9]:
def estandarizar_columnas(df):
    return df.rename(columns={
        'Temperatura': 'temperatura',
        'Humedad': 'humedad',
        'Condicion': 'condicion',
        'Presion': 'presion',
        'Viento': 'viento_velocidad'
    })

## 2.2 Imputa clima por estacion

In [10]:
def imputar_clima_estacion(
    df_clima,
    df_referencia,
    col_fecha_clima='Date',
    col_fecha_ref='FL_DATE',
    cols_num=('temperatura', 'presion', 'viento_velocidad'),
    col_condicion='condicion',
    col_humedad='humedad'
):
    """
    Imputa valores faltantes en un dataframe de clima usando:
    - interpolación temporal
    - estadísticas mensuales propias
    - estadísticas mensuales de un dataframe de referencia
    """

    df = df_clima.copy()

    # --- Fechas a datetime ---
    df[col_fecha_clima] = pd.to_datetime(df[col_fecha_clima])

    if col_fecha_ref not in df_referencia.columns:
        df_referencia[col_fecha_ref] = pd.to_datetime(df_referencia[col_fecha_ref])

    # --- Crear MONTH ---
    df['MONTH'] = df[col_fecha_clima].dt.month

    if 'MONTH' not in df_referencia.columns:
        df_referencia = df_referencia.copy()
        df_referencia['MONTH'] = df_referencia[col_fecha_ref].dt.month

    # --- Imputación numérica ---
    for col in cols_num:
        df[col] = df[col].interpolate(method='linear')
        df[col] = df[col].fillna(
            df.groupby('MONTH')[col].transform('median')
        )

    # --- Imputación de condición (moda mensual) ---
    def get_mode(x):
        m = x.mode()
        return m.iloc[0] if not m.empty else "Normal"

    df[col_condicion] = df.groupby('MONTH')[col_condicion].transform(
        lambda x: x.fillna(get_mode(x))
    )

    # --- Humedad desde dataset de referencia ---
    dict_humedad_mensual = (
        df_referencia
        .groupby('MONTH')[col_humedad]
        .median()
        .to_dict()
    )

    df[col_humedad] = df[col_humedad].fillna(
        df['MONTH'].map(dict_humedad_mensual)
    )

    return df


## 2.3 Rellena nulos extremos

In [11]:
def rellenar_nulos_extremos(
    df,
    columna,
    metodo_inicio='bfill',
    metodo_final='ffill',
    verbose=True
):
    """
    Rellena nulos al inicio y al final de una serie temporal.
    Primero aplica backfill y luego forward fill.
    """

    df = df.copy()

    # Relleno hacia atrás (inicio del archivo)
    df[columna] = df[columna].fillna(method=metodo_inicio)

    # Relleno hacia adelante (final del archivo)
    df[columna] = df[columna].fillna(method=metodo_final)

    if verbose:
        nulos = df[columna].isna().sum()
        print(f"Nulos finales en {columna}: {nulos}")

    return df


In [12]:
# for col in ['viento_velocidad', 'presion', 'temperatura']:
#     df_11640 = rellenar_nulos_extremos(df_11640, col)

## 2.4 Imputa visibilidad

In [13]:
def imputar_visibilidad_estacion(
    df_estacion,
    modelo,
    col_fecha='Date',
    target='visibilidad'
):
    """
    Imputa visibilidad en una estación usando un modelo entrenado,
    sin sobrescribir valores reales.
    """

    df = df_estacion.copy()

    # Fecha y mes
    df[col_fecha] = pd.to_datetime(df[col_fecha])
    df['MONTH'] = df['MONTH'].astype(int)

    features = [
        'temperatura',
        'humedad',
        'presion',
        'viento_velocidad',
        'condicion',
        'MONTH'
    ]

    # Asegurar columna target
    if target not in df.columns:
        df[target] = pd.NA

    mask = df[target].isna()
    if mask.sum() == 0:
        return df

    X_pred = df.loc[mask, features]

    pred = modelo.predict(X_pred)

    df.loc[mask, target] = pred

    return df


## 2.5 Obtiene IATAS por estacion

In [14]:
def obtener_iatas_por_estacion(
    airport_station_mapping,
    station_id
):
    """
    Devuelve la lista de códigos IATA asociados a una estación climática.
    """
    return (
        airport_station_mapping[
            airport_station_mapping['STATION_ID'].astype(str) == str(station_id)
        ]['IATA']
        .dropna()
        .unique()
        .tolist()
    )


## 2.6 Expande clima por IATA

In [15]:
def expandir_clima_por_iata(
    df_clima,
    lista_iatas,
    col_iata='ORIGIN'
):
    """
    Duplica el dataframe de clima una vez por cada IATA
    y asigna el código de aeropuerto correspondiente.
    """
    dfs = []

    for iata in lista_iatas:
        temp_df = df_clima.copy()
        temp_df[col_iata] = iata
        dfs.append(temp_df)

    return pd.concat(dfs, ignore_index=True)


## 2.7 Prepara la esatción para merge

In [16]:
def preparar_df_estacion_para_merge(
    df_estacion,
    col_fecha_estacion='Date',
    col_origen='ORIGIN',
    columnas_clima=(
        'temperatura',
        'humedad',
        'presion',
        'visibilidad',
        'viento_velocidad',
        'condicion'
    )
):
    """
    Prepara el dataframe de una estación climática
    para ser unido con df_procesado.
    """

    df = df_estacion.copy()

    # Fecha a datetime
    df[col_fecha_estacion] = pd.to_datetime(df[col_fecha_estacion])

    # Nos quedamos solo con las columnas necesarias que existan
    cols_disponibles = [
        c for c in columnas_clima
        if c in df.columns
    ]

    cols_finales = [col_origen, col_fecha_estacion] + cols_disponibles

    return df[cols_finales]


## 2.8 Añade estacion a procesado

In [17]:
def añadir_estacion_a_procesado(
    df_procesado,
    df_estacion,
    col_fecha_proc='FL_DATE',
    col_fecha_estacion='Date',
    col_origen='ORIGIN',
    columnas_a_rellenar=(
        'temperatura',
        'humedad',
        'presion',
        'visibilidad',
        'viento_velocidad',
        'condicion'
    )
):
    """
    Añade información climática de una estación al df_procesado,
    rellenando únicamente los valores NaN.
    """

    df_proc = df_procesado.copy()

    # Asegurar datetime
    df_proc[col_fecha_proc] = pd.to_datetime(df_proc[col_fecha_proc])

    # Merge auxiliar
    df_merge = df_proc.merge(
        df_estacion,
        how='left',
        left_on=[col_origen, col_fecha_proc],
        right_on=[col_origen, col_fecha_estacion],
        suffixes=('', '_est')
    )

    # Rellenar solo NaN
    for col in columnas_a_rellenar:
        col_est = f"{col}_est"
        if col in df_merge.columns and col_est in df_merge.columns:
            df_merge[col] = df_merge[col].fillna(df_merge[col_est])

    # Limpiar columnas auxiliares
    cols_a_eliminar = [c for c in df_merge.columns if c.endswith('_est')]
    df_merge.drop(columns=cols_a_eliminar, inplace=True)

    # Eliminar columna Date si quedó
    if col_fecha_estacion in df_merge.columns:
        df_merge.drop(columns=[col_fecha_estacion], inplace=True)

    return df_merge


## 2.9 Verifica nulos y cobertura de clima

In [18]:
def verificar_nulos_y_cobertura_clima(
    df_procesado,
    col_origen='ORIGIN',
    columnas_clima=(
        'temperatura',
        'humedad',
        'presion',
        'visibilidad',
        'viento_velocidad',
        'condicion'
    ),
    devolver_aeropuertos=True
):
    """
    Verifica:
    1) Cantidad de valores nulos por variable climática
    2) Aeropuertos que aún no tienen datos de clima
    """

    # --- Nulos por variable ---
    nulos_por_variable = (
        df_procesado[list(columnas_clima)]
        .isna()
        .sum()
        .sort_values(ascending=False)
    )

    # --- Aeropuertos sin clima ---
    df_clima = df_procesado[[col_origen] + list(columnas_clima)]

    # Un aeropuerto se considera "sin clima" si TODAS las variables están nulas
    aeropuertos_sin_clima = (
        df_clima
        .groupby(col_origen)
        .apply(lambda x: x[list(columnas_clima)].isna().all().all())
    )

    aeropuertos_sin_clima = aeropuertos_sin_clima[aeropuertos_sin_clima].index.tolist()

    resultado = {
        'nulos_por_variable': nulos_por_variable,
        'cantidad_aeropuertos_sin_clima': len(aeropuertos_sin_clima)
    }

    if devolver_aeropuertos:
        resultado['aeropuertos_sin_clima'] = aeropuertos_sin_clima

    return resultado


# 3. Estado inicial de df_procesado

In [19]:
resultado = verificar_nulos_y_cobertura_clima(df_procesado)

print("Nulos por variable:")
print(resultado['nulos_por_variable'])

print("\nCantidad de aeropuertos sin clima:")
print(resultado['cantidad_aeropuertos_sin_clima'])

print("\nAeropuertos sin clima:")
print(resultado['aeropuertos_sin_clima'])

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3432694235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[list(columnas_clima)].isna().all().all())


Nulos por variable:
temperatura         876108
humedad             876108
presion             876108
visibilidad         876108
viento_velocidad    876108
condicion           876108
dtype: int64

Cantidad de aeropuertos sin clima:
55

Aeropuertos sin clima:
['ACY', 'ANC', 'ASE', 'ATW', 'AUS', 'AZA', 'AZO', 'BGR', 'BLV', 'BMI', 'BQN', 'CLD', 'COD', 'DAB', 'DCA', 'DEC', 'DVL', 'ECP', 'ESC', 'FLG', 'GCC', 'GPT', 'GST', 'GUC', 'GUM', 'HNL', 'HOB', 'HYS', 'IMT', 'ITH', 'LAW', 'LRD', 'MHK', 'MSP', 'ORD', 'OTZ', 'PAE', 'PBG', 'PPG', 'PSC', 'PSE', 'PSP', 'RHI', 'RIW', 'SAF', 'SGU', 'SJU', 'SLN', 'SPN', 'STT', 'STX', 'TWF', 'WRG', 'XNA', 'XWA']


# 4. Estación 99999 ASPEN

## 4.1 Renombra variables

In [20]:
df_99999 = estandarizar_columnas(df_99999)

In [21]:
df_99999.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad
0,2024-10-01,57.0,NaN,NaN,22.85,6.6
1,2024-10-02,56.0,NaN,NaN,22.73,7.4
2,2024-10-03,54.0,NaN,NaN,22.73,6.7


In [22]:
df_99999.shape

(328, 6)

In [23]:
df_99999.isna().sum()

Date                  0
temperatura           1
humedad             328
condicion           182
presion               5
viento_velocidad      1
dtype: int64

## 4.2 Sustituye nan

In [24]:
df_99999 = imputar_clima_estacion(
    df_clima=df_99999,
    df_referencia=df_procesado
)

In [25]:
df_99999.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
dtype: int64

## 4.3 Incluye variable visibilidad

In [26]:
df_99999.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH
0,2024-10-01,57.0,64.0,RA,22.85,6.6,10


In [27]:
df_99999 = imputar_visibilidad_estacion(
    df_estacion=df_99999,
    modelo=modelo_visibilidad
)

In [28]:
df_99999.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad
0,2024-10-01,57.0,64.0,RA,22.85,6.6,10,16.0591


## 4.4 Incluye aeropuerto asociado

In [29]:
# Estación 99999
iatas_99999 = obtener_iatas_por_estacion(
    airport_station_mapping,
    station_id=99999
)

df_99999 = expandir_clima_por_iata(
    df_99999,
    iatas_99999
)

In [30]:
df_99999.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad,ORIGIN
0,2024-10-01,57.0,64.0,RA,22.85,6.6,10,16.0591,ASE
1,2024-10-02,56.0,64.0,RA,22.73,7.4,10,16.047133,ASE
2,2024-10-03,54.0,64.0,RA,22.73,6.7,10,16.06,ASE


In [31]:
df_99999.ORIGIN.unique()

array(['ASE', 'ATW', 'AUS', 'AZO', 'BGR', 'BLV', 'BMI', 'COD', 'CLD',
       'DEC', 'DVL', 'ECP', 'ESC', 'FLG', 'GCC', 'GPT', 'GUC', 'HOB',
       'HYS', 'IMT', 'ITH', 'AZA', 'LAW', 'MHK', 'PAE', 'PSC', 'PSP',
       'RHI', 'RIW', 'SAF', 'SGU', 'TWF', 'XNA', 'GST', 'ANC', 'OTZ',
       'WRG', 'HNL', 'PSE'], dtype=object)

In [32]:
len(df_99999.ORIGIN.unique())

39

In [33]:
df_99999.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
visibilidad         0
ORIGIN              0
dtype: int64

## 4.5 Incluye estación a procesado

In [34]:
df_99999_preparado = preparar_df_estacion_para_merge(df_99999)

df_procesado = añadir_estacion_a_procesado(
    df_procesado,
    df_99999_preparado
)

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3926363365.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merge[col] = df_merge[col].fillna(df_merge[col_est])


## 4.6 Estado de df_procesado después de añadir estación

In [35]:
resultado = verificar_nulos_y_cobertura_clima(df_procesado)

print("Nulos por variable:")
print(resultado['nulos_por_variable'])

print("\nCantidad de aeropuertos sin clima:")
print(resultado['cantidad_aeropuertos_sin_clima'])

print("\nAeropuertos sin clima:")
print(resultado['aeropuertos_sin_clima'])


C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3432694235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[list(columnas_clima)].isna().all().all())


Nulos por variable:
temperatura         640787
humedad             640787
presion             640787
visibilidad         640787
viento_velocidad    640787
condicion           640787
dtype: int64

Cantidad de aeropuertos sin clima:
16

Aeropuertos sin clima:
['ACY', 'BQN', 'DAB', 'DCA', 'GUM', 'LRD', 'MSP', 'ORD', 'PBG', 'PPG', 'SJU', 'SLN', 'SPN', 'STT', 'STX', 'XWA']


# 5. Estación 11640 CYRIL

In [36]:
df_11640.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad
0,2024-10-01,85.0,NaN,NaN,29.85,NaN
1,2024-10-02,83.0,NaN,RA BR,29.83,NaN
2,2024-10-03,86.0,NaN,NaN,29.84,NaN


In [37]:
df_11640.isna().sum()

Date                  0
temperatura           9
humedad             319
condicion           151
presion              28
viento_velocidad     58
dtype: int64

## 5.1 Sustituye nan

In [38]:
df_11640 = imputar_clima_estacion(
    df_clima=df_11640,
    df_referencia=df_procesado
)

In [39]:
df_11640.isna().sum()

Date                 0
temperatura          0
humedad              0
condicion            0
presion              0
viento_velocidad    30
MONTH                0
dtype: int64

In [40]:
print(f"Rango de viento: {df_11640['viento_velocidad'].min()} a {df_11640['viento_velocidad'].max()}")
print(f"Nulos finales: {df_11640['viento_velocidad'].isna().sum()}")

Rango de viento: 3.0 a 18.5
Nulos finales: 30


## 5.2 Sustituye nulos extremos

In [41]:
df_11640 = rellenar_nulos_extremos(
    df=df_11640,
    columna='viento_velocidad'
)

Nulos finales en viento_velocidad: 0


C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\2546991623.py:16: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[columna] = df[columna].fillna(method=metodo_inicio)
C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\2546991623.py:19: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[columna] = df[columna].fillna(method=metodo_final)


In [42]:
df_11640.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
dtype: int64

## 5.3 Incluye variable visibilidad

In [43]:
df_11640.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH
0,2024-10-01,85.0,64.0,RA,29.85,9.8,10


In [44]:
df_11640 = imputar_visibilidad_estacion(
    df_estacion=df_11640,
    modelo=modelo_visibilidad
)

In [45]:
df_11640.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad
0,2024-10-01,85.0,64.0,RA,29.85,9.8,10,16.0561


## 5.4 Incluye aeropuerto asociado

In [46]:
iatas_11640 = obtener_iatas_por_estacion(
    airport_station_mapping,
    station_id=11640
)

df_11640 = expandir_clima_por_iata(
    df_11640,
    iatas_11640
)

In [47]:
df_11640.ORIGIN.unique()

array(['STT'], dtype=object)

In [57]:
df_11640.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad,ORIGIN
0,2024-10-01,85.0,64.0,RA,29.85,9.8,10,16.0561,STT
1,2024-10-02,83.0,64.0,RA BR,29.83,9.8,10,16.0597,STT
2,2024-10-03,86.0,64.0,RA,29.84,9.8,10,16.0561,STT


In [58]:
df_11640.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
visibilidad         0
ORIGIN              0
dtype: int64

## 5.5 Incluye estación a procesado

In [59]:
df_11640_preparado = preparar_df_estacion_para_merge(df_11640)

df_procesado = añadir_estacion_a_procesado(
    df_procesado,
    df_11640_preparado
)

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3926363365.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merge[col] = df_merge[col].fillna(df_merge[col_est])


## 5.6 Estado de df_procesado despues de añadir estacion

In [60]:
resultado = verificar_nulos_y_cobertura_clima(df_procesado)

print("Nulos por variable:")
print(resultado['nulos_por_variable'])

print("\nCantidad de aeropuertos sin clima:")
print(resultado['cantidad_aeropuertos_sin_clima'])

print("\nAeropuertos sin clima:")
print(resultado['aeropuertos_sin_clima'])

Nulos por variable:
temperatura         636189
humedad             636189
presion             636189
visibilidad         636189
viento_velocidad    636189
condicion           636189
dtype: int64

Cantidad de aeropuertos sin clima:
15

Aeropuertos sin clima:
['ACY', 'BQN', 'DAB', 'DCA', 'GUM', 'LRD', 'MSP', 'ORD', 'PBG', 'PPG', 'SJU', 'SLN', 'SPN', 'STX', 'XWA']


C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3432694235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[list(columnas_clima)].isna().all().all())


# 6. Estación 12846 DAYTONA

In [50]:
df_12846.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad
0,2024-10-1,84.0,NaN,TS RA BR,29.87,3.7
1,2024-10-2,82.0,NaN,RA,29.86,7.7
2,2024-10-3,82.0,NaN,NaN,29.93,6.0


## 6.1 Sustituye nan

In [51]:
df_12846 = imputar_clima_estacion(
    df_clima=df_12846,
    df_referencia=df_procesado
)

In [52]:
df_12846.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
dtype: int64

## 6.2 Incluye variable visibilidad

In [53]:
df_12846 = imputar_visibilidad_estacion(
    df_estacion=df_12846,
    modelo=modelo_visibilidad
)

In [61]:
df_12846.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad,ORIGIN
0,2024-10-01,84.0,64.0,TS RA BR,29.87,3.7,10,16.055267,DAB


## 6.3 Incluye aeropuerto asociado

In [54]:
# Estación 12846
iatas_12846 = obtener_iatas_por_estacion(
    airport_station_mapping,
    station_id=12846
)

df_12846 = expandir_clima_por_iata(
    df_12846,
    iatas_12846
)

In [55]:
df_12846.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad,ORIGIN
0,2024-10-01,84.0,64.0,TS RA BR,29.87,3.7,10,16.055267,DAB
1,2024-10-02,82.0,64.0,RA,29.86,7.7,10,16.0495,DAB
2,2024-10-03,82.0,64.0,RA BR,29.93,6.0,10,16.0639,DAB


In [56]:
df_12846.ORIGIN.unique()

array(['DAB'], dtype=object)

In [63]:
df_12846.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
visibilidad         0
ORIGIN              0
dtype: int64

## Incluye estacion a procesado

In [62]:
df_12846_preparado = preparar_df_estacion_para_merge(df_12846)

df_procesado = añadir_estacion_a_procesado(
    df_procesado,
    df_12846_preparado
)

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3926363365.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merge[col] = df_merge[col].fillna(df_merge[col_est])


## 6.4 Estado de df_procesado despues de añadir estacion

In [64]:
resultado = verificar_nulos_y_cobertura_clima(df_procesado)

print("Nulos por variable:")
print(resultado['nulos_por_variable'])

print("\nCantidad de aeropuertos sin clima:")
print(resultado['cantidad_aeropuertos_sin_clima'])

print("\nAeropuertos sin clima:")
print(resultado['aeropuertos_sin_clima'])

Nulos por variable:
temperatura         633554
humedad             633554
presion             633554
visibilidad         633554
viento_velocidad    633554
condicion           633554
dtype: int64

Cantidad de aeropuertos sin clima:
14

Aeropuertos sin clima:
['ACY', 'BQN', 'DCA', 'GUM', 'LRD', 'MSP', 'ORD', 'PBG', 'PPG', 'SJU', 'SLN', 'SPN', 'STX', 'XWA']


C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3432694235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[list(columnas_clima)].isna().all().all())


# 7. Estación 41415 GUAM 

In [66]:
df_41415.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad
0,2024-10-1,85.0,NaN,RA BR,NaN,9.7


## 7.1 Sustituye nan

In [67]:
df_41415.isna().sum()

Date                  0
temperatura           1
humedad             329
condicion            92
presion               5
viento_velocidad      1
dtype: int64

In [68]:
df_41415 = imputar_clima_estacion(
    df_clima=df_41415,
    df_referencia=df_procesado
)

In [69]:
df_41415.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
dtype: int64

## 7.2 Incluye variable visibilidad

In [70]:
df_41415 = imputar_visibilidad_estacion(
    df_estacion=df_41415,
    modelo=modelo_visibilidad
)

In [71]:
df_41415.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad
0,2024-10-01,85.0,64.0,RA BR,29.5475,9.7,10,16.0597


## 7.3 Incluye aeropuerto asociado

In [72]:
# Estación 41415
iatas_41415 = obtener_iatas_por_estacion(
    airport_station_mapping,
    station_id=41415
)

df_41415 = expandir_clima_por_iata(
    df_41415,
    iatas_41415
)

In [73]:
df_41415.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad,ORIGIN
0,2024-10-01,85.0,64.0,RA BR,29.5475,9.7,10,16.0597,GUM


## 7.4 Incluye estacion a procesado

In [74]:
df_41415_preparado = preparar_df_estacion_para_merge(df_41415)

df_procesado = añadir_estacion_a_procesado(
    df_procesado,
    df_41415_preparado
)

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3926363365.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merge[col] = df_merge[col].fillna(df_merge[col_est])


## 7.5 Estado de df_procesado despues de añadir estacion

In [75]:
resultado = verificar_nulos_y_cobertura_clima(df_procesado)

print("Nulos por variable:")
print(resultado['nulos_por_variable'])

print("\nCantidad de aeropuertos sin clima:")
print(resultado['cantidad_aeropuertos_sin_clima'])

print("\nAeropuertos sin clima:")
print(resultado['aeropuertos_sin_clima'])

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3432694235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[list(columnas_clima)].isna().all().all())


Nulos por variable:
temperatura         632898
humedad             632898
presion             632898
visibilidad         632898
viento_velocidad    632898
condicion           632898
dtype: int64

Cantidad de aeropuertos sin clima:
13

Aeropuertos sin clima:
['ACY', 'BQN', 'DCA', 'LRD', 'MSP', 'ORD', 'PBG', 'PPG', 'SJU', 'SLN', 'SPN', 'STX', 'XWA']


# 8. Estación 11624 HENRY

In [77]:
df_11624.head(3)

,Date,temperatura,humedad,condicion,presion,viento_velocidad
0,2024-10-1,84.0,NaN,RA,29.82,5.8
1,2024-10-2,83.0,NaN,TS RA BR,29.79,4.3
2,2024-10-3,85.0,NaN,RA BR,29.79,5.8


## 8.1 Sustituye nan

In [78]:
df_11624.isna().sum()

Date                  0
temperatura          26
humedad             325
condicion           160
presion              13
viento_velocidad      5
dtype: int64

In [79]:
df_11624 = imputar_clima_estacion(
    df_clima=df_11624,
    df_referencia=df_procesado
)

In [80]:
df_11624.isna().sum()

Date                0
temperatura         0
humedad             0
condicion           0
presion             0
viento_velocidad    0
MONTH               0
dtype: int64

## 8.2 Incluye variable visibilidad

In [81]:
df_11624 = imputar_visibilidad_estacion(
    df_estacion=df_11624,
    modelo=modelo_visibilidad
)

In [82]:
df_11624.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad
0,2024-10-01,84.0,64.0,RA,29.82,5.8,10,16.0603


## 8.3 Incluye aeropuerto asociado

In [83]:
# Estación 11624
iatas_11624 = obtener_iatas_por_estacion(
    airport_station_mapping,
    station_id=11624
)

df_11624 = expandir_clima_por_iata(
    df_11624,
    iatas_11624
)

In [84]:
df_11624.head(1)

,Date,temperatura,humedad,condicion,presion,viento_velocidad,MONTH,visibilidad,ORIGIN
0,2024-10-01,84.0,64.0,RA,29.82,5.8,10,16.0603,STX


## 8.4 Incluye estacion a procesado

In [85]:
df_11624_preparado = preparar_df_estacion_para_merge(df_11624)

df_procesado = añadir_estacion_a_procesado(
    df_procesado,
    df_11624_preparado
)

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3926363365.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merge[col] = df_merge[col].fillna(df_merge[col_est])


## 8.5 Estado de df_procesado despues de añadir estacion

In [86]:
resultado = verificar_nulos_y_cobertura_clima(df_procesado)

print("Nulos por variable:")
print(resultado['nulos_por_variable'])

print("\nCantidad de aeropuertos sin clima:")
print(resultado['cantidad_aeropuertos_sin_clima'])

print("\nAeropuertos sin clima:")
print(resultado['aeropuertos_sin_clima'])

C:\Users\Karen\AppData\Local\Temp\ipykernel_15204\3432694235.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x[list(columnas_clima)].isna().all().all())


Nulos por variable:
temperatura         631687
humedad             631687
presion             631687
visibilidad         631687
viento_velocidad    631687
condicion           631687
dtype: int64

Cantidad de aeropuertos sin clima:
12

Aeropuertos sin clima:
['ACY', 'BQN', 'DCA', 'LRD', 'MSP', 'ORD', 'PBG', 'PPG', 'SJU', 'SLN', 'SPN', 'XWA']


# 9. Imputación final de los 12 aeropuertos finales

In [88]:
def imputar_clima_global_por_mes(
    df,
    col_origen='ORIGIN',
    col_mes='MONTH',
    columnas_clima=(
        'temperatura',
        'humedad',
        'presion',
        'visibilidad',
        'viento_velocidad',
        'condicion'
    )
):
    """
    Imputa clima faltante usando estadísticos globales mensuales.
    Pensado para aeropuertos sin ninguna estación asociada.
    """

    df = df.copy()

    # Aeropuertos completamente sin clima
    mask_sin_clima = (
        df.groupby(col_origen)[list(columnas_clima)]
        .transform(lambda x: x.isna().all())
        .all(axis=1)
    )

    # Diccionarios globales por mes
    stats = {}

    for col in columnas_clima:
        if col == 'condicion':
            stats[col] = (
                df[~mask_sin_clima]
                .groupby(col_mes)[col]
                .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
            )
        else:
            stats[col] = (
                df[~mask_sin_clima]
                .groupby(col_mes)[col]
                .median()
            )

    # Imputar
    for col in columnas_clima:
        df.loc[mask_sin_clima, col] = df.loc[mask_sin_clima, col_mes].map(stats[col])

    df.loc[mask_sin_clima, 'clima_global_imputado'] = True
    df['clima_global_imputado'] = df.get('clima_global_imputado', False)

    return df


In [89]:
df_procesado = imputar_clima_global_por_mes(df_procesado)

In [90]:
cols = ['temperatura','humedad','presion','visibilidad','viento_velocidad','condicion']
df_procesado[cols].isna().sum()


temperatura         26613
humedad             26613
presion             26613
visibilidad         26613
viento_velocidad    26613
condicion           26613
dtype: int64

In [91]:
df_procesado[df_procesado['ORIGIN'].isin([
    'ACY','BQN','DCA','LRD','MSP','ORD',
    'PBG','PPG','SJU','SLN','SPN','XWA'
])][cols].head()


,temperatura,humedad,presion,visibilidad,viento_velocidad,condicion
53,22.25,64.0,997.15,16.09,3.35,RA
56,22.25,64.0,997.15,16.09,3.35,RA
64,22.25,64.0,997.15,16.09,3.35,RA
66,22.25,64.0,997.15,16.09,3.35,RA
67,22.25,64.0,997.15,16.09,3.35,RA


## 9.1 Fallback

In [92]:
def imputar_clima_global_fallback(df):
    df = df.copy()

    cols_num = ['temperatura','humedad','presion','visibilidad','viento_velocidad']
    col_cat = 'condicion'

    # Numéricas → mediana global por mes
    for col in cols_num:
        mediana_mes = df.groupby('MONTH')[col].median()
        df[col] = df[col].fillna(df['MONTH'].map(mediana_mes))

    # Categórica → moda global por mes
    def moda_segura(x):
        m = x.mode()
        return m.iloc[0] if not m.empty else 'Normal'

    moda_mes = df.groupby('MONTH')[col_cat].apply(moda_segura)
    df[col_cat] = df[col_cat].fillna(df['MONTH'].map(moda_mes))

    return df


In [93]:
df_procesado = imputar_clima_global_fallback(df_procesado)

In [94]:
cols = ['temperatura','humedad','presion','visibilidad','viento_velocidad','condicion']
df_procesado[cols].isna().sum()


temperatura         0
humedad             0
presion             0
visibilidad         0
viento_velocidad    0
condicion           0
dtype: int64

In [96]:
df_procesado.columns

Index(['QUARTER', 'MONTH', 'DAY_OF_MONTH', 'DAY_OF_WEEK', 'FL_DATE',
       'OP_UNIQUE_CARRIER', 'TAIL_NUM', 'OP_CARRIER_FL_NUM',
       'ORIGIN_AIRPORT_ID', 'ORIGIN', 'DEST_AIRPORT_ID', 'DEST',
       'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'DEP_DEL15', 'CRS_ARR_TIME',
       'ARR_TIME', 'ARR_DELAY', 'ARR_DEL15', 'CANCELLED', 'DIVERTED',
       'ACTUAL_ELAPSED_TIME', 'DISTANCE', 'WEATHER_DELAY',
       'HAS_FIRST_DEP_TIME', 'HAS_TOTAL_ADD_GTIME', 'FECHA_PARTIDA', 'Hour',
       'temperatura', 'humedad', 'presion', 'visibilidad', 'viento_velocidad',
       'condicion', 'clima_global_imputado'],
      dtype='object')

In [97]:
df_procesado.drop(columns=['Hour', 'clima_global_imputado'], inplace=True)

# 10. Guardado

In [98]:
df_procesado.to_csv('df_procesado_clima.csv', index=False, encoding='utf-8')

In [99]:
df_listo = pd.read_csv('df_procesado_clima.csv')

In [100]:
df_listo.isna().sum()

QUARTER                0
MONTH                  0
DAY_OF_MONTH           0
DAY_OF_WEEK            0
FL_DATE                0
OP_UNIQUE_CARRIER      0
TAIL_NUM               0
OP_CARRIER_FL_NUM      0
ORIGIN_AIRPORT_ID      0
ORIGIN                 0
DEST_AIRPORT_ID        0
DEST                   0
CRS_DEP_TIME           0
DEP_TIME               0
DEP_DELAY              0
DEP_DEL15              0
CRS_ARR_TIME           0
ARR_TIME               0
ARR_DELAY              0
ARR_DEL15              0
CANCELLED              0
DIVERTED               0
ACTUAL_ELAPSED_TIME    0
DISTANCE               0
WEATHER_DELAY          0
HAS_FIRST_DEP_TIME     0
HAS_TOTAL_ADD_GTIME    0
FECHA_PARTIDA          0
temperatura            0
humedad                0
presion                0
visibilidad            0
viento_velocidad       0
condicion              0
dtype: int64